<a href="https://colab.research.google.com/github/phisan-chula/2021-LDP_Design/blob/main/LDP_Inspection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Design of a Low Distortion Projection for Mini Project**
Phisan Santitamonont, Chulalongkorn University 2022
Phisan.Chula@gmail.com

In [47]:
# %%capture
! pip install pygeodesy
! pip install pyproj
! pip install requests
! pip install folium

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [48]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [49]:
import requests
import pygeodesy as pgd
import numpy as np
from shapely.geometry import Point

def dd2DMS( dd, PREC=7, POS=''  ):
    '''conver degree to DMS string'''
    return pgd.dms.toDMS( dd, prec=PREC,pos=POS )
def CalcLDP( row ):
    UNDUL = GEOID.height( row.lat,row.lng )
    RG = ELLPS.rocGauss( row.lat )
    h = UNDUL + row.MSL     # FIX UNDUL
    HSF = RG/(RG+h)            # FIX  RG
    FACTOR = pyproj.Proj( LDP).get_factors( row.lng, row.lat )
    PSF = FACTOR.meridional_scale
    CSF = PSF*HSF
    CSF_ppm = (CSF-1)*1E6
    TR = pyproj.Transformer.from_crs( 'epsg:4326', LDP )
    LDP_E,LDP_N = TR.transform( row.lat, row.lng )
    return [UNDUL, h, HSF, PSF, CSF, CSF_ppm, LDP_E, LDP_N]

In [59]:
# download tgm2017 geoid model to you own
TGM17 = r'/content/drive/MyDrive/Geodesy_Cache/tgm2017-1.pgm'

POS =[13.5411528, 99.8240430]  # origin point @ lat,lng and topo(MSL) height
HOR_BUF = 2_000  #meter   buffer of the PP but  a rectangle
VER_BUF = 20        # meter  + and -

In [60]:
###################################################
res = requests.get( r'https://api.opentopodata.org/v1/srtm30m?locations={},{}'.format( *POS ) )
MSL = res.json()['results'][0]['elevation']
####################################################
GEOID = pgd.geoids.GeoidKarney( TGM17 )
UNDUL = GEOID.height( *POS )
HAE = UNDUL + MSL
print( f'Projection Plane : lat = {dd2DMS(POS[0]):}  lng = {dd2DMS(POS[1]):}  ')
print( f'MSL = {MSL:.1f} m. HAE={HAE:.1f} m. ,  N={UNDUL:.1f} m.' )
ELLPS  = pgd.datums.Ellipsoids.WGS84
RG = ELLPS.rocGauss( POS[0] )

Projection Plane : lat = 13°32′28.15008″  lng = 99°49′26.5548″  
MSL = 5.0 m. HAE=-26.9 m. ,  N=-31.9 m.


In [64]:
import pandas as pd
import pyproj
COL_LDP = ['UNDUL', 'h','HSF','PSF','CSF', 'CSF_ppm', 'LDP_E', 'LDP_N']
k0 = 1 + HAE/RG
TM = r'+proj=tmerc +lat_0=0.0 +lon_0={} +k_0={}  +x_0={}  +y_0={}  +a={} +b={} +units=m +no_defs'
LDP = pyproj.CRS( TM.format( 99+49/60, k0, +2500, -1490000, ELLPS.a, ELLPS.b ) )  # fine tune , serveral times

EWNS =Point( POS[1],POS[0] ).buffer( BUFF/ 111_000 , cap_style = 3  ).exterior.coords.xy
PP_EWNS = np.vstack( (np.array( [POS[1],POS[0]] ) , np.array(EWNS).T) )[:-1]
dfPP = pd.DataFrame( {'Point':['P0','P1','P2','P3','P4'], 'lng':PP_EWNS[:,0], 'lat':PP_EWNS[:,1]  } )
dfPP = pd.concat( 2*[dfPP] ,  ignore_index=True)
dfPP['MSL'] = 5*[MSL+VER_BUF]+ 5*[MSL-VER_BUF]
dfPP[COL_LDP] = dfPP.apply( CalcLDP, axis=1, result_type='expand')
dfPP

,Point,lng,lat,MSL,UNDUL,h,HSF,PSF,CSF,CSF_ppm,LDP_E,LDP_N
0,P0,99.824043,13.541153,25.0,-31.905258,-6.905258,1.000001,0.999996,0.999997,-3.137183,3298.447062,7573.740610
1,P1,99.842061,13.559171,25.0,-31.880289,-6.880289,1.000001,0.999996,0.999997,-3.055580,5248.590379,9567.290979
2,P2,99.842061,13.523135,25.0,-31.831244,-6.831244,1.000001,0.999996,0.999997,-3.063263,5249.004117,5580.454299
3,P3,99.806025,13.523135,25.0,-31.930603,-6.930603,1.000001,0.999996,0.999997,-3.124669,1348.010178,5580.336861
4,P4,99.806025,13.559171,25.0,-31.998387,-6.998387,1.000001,0.999996,0.999997,-3.114016,1348.183558,9567.173251
5,P0,99.824043,13.541153,-15.0,-31.905258,-46.905258,1.000007,0.999996,1.000003,3.153056,3298.447062,7573.740610
6,P1,99.842061,13.559171,-15.0,-31.880289,-46.880289,1.000007,0.999996,1.000003,3.234653,5248.590379,9567.290979
7,P2,99.842061,13.523135,-15.0,-31.831244,-46.831244,1.000007,0.999996,1.000003,3.226982,5249.004117,5580.454299
8,P3,99.806025,13.523135,-15.0,-31.930603,-46.930603,1.000007,0.999996,1.000003,3.165576,1348.010178,5580.336861
9,P4,99.806025,13.559171,-15.0,-31.998387,-46.998387,1.000007,0.999996,1.000003,3.176217,1348.183558,9567.173251


In [65]:
from io import StringIO
CSV = '''Point,lat,lng,MSL
RTK1,13.542153,99.824043,20.0
RTK2,13.540153,99.821043,-30.0
RTK3,13.541153,99.822043,2.0
'''
dfRTK = pd.read_csv( StringIO(CSV) )
dfRTK[COL_LDP] = dfRTK.apply( CalcLDP, axis=1, result_type='expand')
dfRTK = dfRTK.round( {'UNDUL':1, 'h':1, 'CSF_ppm':1 , 'LDP_E':3, 'LDP_N':3  } )
dfRTK

,Point,lat,lng,MSL,UNDUL,h,HSF,PSF,CSF,CSF_ppm,LDP_E,LDP_N
0,RTK1,13.542153,99.824043,20.0,-31.9,-11.9,1.000002,0.999996,0.999998,-2.4,3298.444,7684.397
1,RTK2,13.540153,99.821043,-30.0,-31.9,-61.9,1.000010,0.999996,1.000006,5.5,2973.716,7463.120
2,RTK3,13.541153,99.822043,2.0,-31.9,-29.9,1.000005,0.999996,1.000000,0.5,3081.958,7573.757


In [54]:
from pandas.io.formats.printing import default_pprint
import folium
icon=folium.Icon(color='red', icon='plus' )
map = folium.Map(location =[POS[0],POS[1]],  zoom_start = 15 )
for i,row in dfPP.iterrows():
    folium.CircleMarker(location=[row.lat, row.lng],  radius=10,   
                color='blue',  fill_color ='blue',  fill_opacity=0.5 ).add_to(map)
for i,row in dfRTK.iterrows():
    folium.CircleMarker(location=[row.lat, row.lng],  radius=10,   
                color='red',  fill_color ='red',  fill_opacity=0.5 ).add_to(map)
map